# TP 1: Análisis Exploratorio de Datos

**Integrantes:**
- Cejas, Victoria
- Festa, Georgina

**Fecha de entrega:** 1 de Mayo del 2020

## Información sobre el dataset:
*El dataset que se va a trabajar durante este trabajo práctico está compuesto por 142 columnas y 2.600.000 filas. El mismo contiene datos historicos de préstamos emitidos en el periodo 2010-2018*

Del dataset original se selecciono un subconjunto de variables, ya que el mismo era de gran tamaño, lo cual volvía muy complejo el modelo. Sumado a esto, se hizo un análisis de los datos en función al taget elegido para así poder seleccionar aquellas variables consideradas relevantes. 
Esto dio como resultado un nuevo conjunto de datos conformado por 23 columnas y 1.300.000 filas.

**Sobre el tratamiento de las filas:**
Se eliminaron aquellas filas donde el valor de la variable de la columna "load_status" era diferente de fully paid, charge off o default ya que las mismas no podían ser clasificadas de forma que la variable de salida sea binaria.

## Diccionario de datos
 
| Variable      |Definición                                                                | ¿Es utilizada? | Entrada/Salida |
|:--------------|:-------------------------------------------------------------------------|:--------------|:----------------|
| acc_now_delinq | Número de cuentas en el que el prestatario está en mora.               | SI           | ENTRADA              |
| annual_inc  | Ingreso anual informado proporcionado por el prestatario.              | SI           | ENTRADA              |
| delinq_amnt  | Monto adeudado por las cuentas que el prestatario está en mora actualmente.                | SI           | ENTRADA              |
| disbursement_method  | Método por el cual el pestatario recibe su préstamo. Puede ser CASH (efectivo), DIRECT PAY (pago directo).                 | NO           | -              |
| dti  | Relación entre los pagos mensuales totales de la deuda dividido por el ingreso mensual informado del prestatario.                 | SI           | ENTRADA              |
| earliest_cr_line  | Mes en el que se abrió la primera línea de préstamo informada por el prestatario                 | NO           | -              |
| emp_length  | Duración del empleo en años.                  | SI           | ENTRADA             |
| emp_title  | El título del trabajo proporcionado por el prestatario al solicitar el préstamo.                 | NO           | -              |
| funded_amnt  | Monto total comprometido con el préstamo hasta el momento.                 | SI           | ENTRADA              |
| grade  | Tipo de préstamo al que puede acceder el prestatario en función a su perfil de deuda.                 | SI           | ENTRADA              |
| home_ownership  | El estado de propiedad de la vivienda del prestatario.                  | SI           | ENTRADA              |
| installment  | Pago/cuota mensual del préstamo.                 | SI           | ENTRADA              |
| int_rate  | Tasa de interés del préstamo.                | SI           | ENTRADA              |
| issue_d  | Mes en el que se financió el préstamo.                 | NO           | -              |
| last_pymnt_amnt  | Monto del último pago recibido.                 | NO           | -              |
| loan_status   | Estado actual del préstamo.                 | SI           | SALIDA              |
| open_acc  | Cantidad de créditos activos que posee el prestatario               | NO           | -              |
| purpose  | Propósito del préstamo definido por el préstatario para la solicitud de préstamo                | SI           | ENTRADA              |
| term  | Número de meses de pago del préstamo.                 | SI           | ENTRADA              |
| title  | El título del préstamo proporcionado por el prestatario.                | NO           | -              |
| tot_coll_amt  | Monto total de cobro adeudado.                 | NO           | -              |
| total_bal_ex_mort  | Saldo total del crédito excluyendo la hipoteca.               | NO           | -              |
| total_pymnt  | Pagos recibidos hasta la fecha.               | SI           | ENTRADA              |

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sb
import itertools
import warnings
import plotly.express as px
from sklearn.preprocessing import minmax_scale
warnings.filterwarnings("ignore")

In [ ]:
#Importar csv
original_data=pd.read_csv('dataset/loans_data.csv')

In [ ]:
#Datos originales
original_data.head()

## Variable de salida:
La variable de salida **"loan_status"** puede tener los siguientes valores:
 - **Fully paid (Totalmente pagado):** En el caso de que el préstamo haya sido pagado en su totalidad.
 - **No paid (No pagado):** En el caso de que el préstamo no se haya pagado en su totalidad
 
Originalmente en el dataset está variable podía contener alguno de los siguientes valores:
- Current
- Fully Paid
- Late (31 - 120 days)
- In Grace Period
- Charged Off
- Late (16-30 days)

Pero decidimos simplificar los valores de la variable target porque consideramos que los valores elegidos se adaptan mejor a nuestro modelo, ya que *lo que queremos predecir es si un préstamo se pagó o no se pagó.*

In [ ]:
#Quitar status de loan que no nos interesan

new_data=original_data.drop(original_data[original_data['loan_status']=='Current'].index)

new_data=new_data.drop(new_data[new_data['loan_status']=='Late (16-30 days)'].index)
new_data=new_data.drop(new_data[new_data['loan_status']=='Late (31-120 days)'].index)
new_data=new_data.drop(new_data[new_data['loan_status']=='In Grace Period'].index)
new_data=new_data.drop(new_data[new_data['loan_status']=='Does not meet the credit policy. Status:Fully Paid'].index)
new_data=new_data.drop(new_data[new_data['loan_status']=='Does not meet the credit policy. Status:Charged Off'].index)


In [ ]:
#volver Target Binario
new_data=new_data.replace(regex=['Default','Charged Off'],value='No paid')

## Transformaciones para variables de entrada: Limpieza y pre-procesado

### Valores nulos:

In [ ]:
#Total de nulos
new_data.isnull().sum()

In [ ]:
#Quita nulos
new_data= new_data.dropna()

In [ ]:
#Se cree nuevo dataset 'process_data' con los datos procesados
process_data=new_data.copy()

## Variables no utilizadas:
- Variables : id, member_id, url, desc:
    Estas variables no son utilizadas porque contienen valores nulos.
    
    Además, variables como id, o memeber_id llevarían a que el modelo se sobreentrene produciendo así overfitting, lo cual representa un problema a la hora de predecir.
    
    
- Demás variables:
    No son utilizadas porque:
        - no tienen relevancia en la predicción de la variable de salida.
        - son redundantes (Ej.: title y purpose)

In [ ]:
#Quita columnas
new_data=new_data.drop(['emp_title','title','disbursement_method','earliest_cr_line','issue_d','last_pymnt_amnt','open_acc','total_bal_ex_mort','tot_coll_amt'],axis=1)

### Pre-Procesamiento: Escalar

In [ ]:
#Escalar:funded_amnt,int_rate,installment,annual_inc,total_pymnt,total_coll_amnt,delinq_amnt,dti
process_data[['funded_amnt','int_rate','installment','annual_inc','total_pymnt','delinq_amnt','dti']]= minmax_scale(new_data[['funded_amnt','int_rate','installment','annual_inc','total_pymnt','delinq_amnt','dti']])

### Pre-procesamiento: One-hot encoder de la variable "purpose":

In [ ]:
process_data.purpose.unique()

In [ ]:
#One Hot-Ecoder Variable: Purpose

for purpose in process_data.purpose.unique():
    process_data[purpose.lower().replace(" ","_")]= (process_data.purpose==purpose)

process_data=process_data.drop(['purpose'],axis=1)



In [ ]:
pd.set_option('display.max_columns',None)
process_data.head()


In [ ]:
#Datos a utilizar
new_data.head()

In [ ]:
#Tipo de datos de cada columna
new_data.dtypes

## Análisis de la variable de salida:
### Balanceo:

In [ ]:
#Datos de salida Balanceo de datos

import matplotlib.pyplot as plt
import numpy as np

x=new_data.loan_status.unique()
y=new_data.loan_status.value_counts()

colors = ['yellowgreen','blue','gold','lightskyblue','coral','grey','pink', 'green','violet']
porcent = 100.*y/y.sum()

patches, texts = plt.pie(y, colors=colors, startangle=90, radius=3)
labels = ['{0} - {1:1.2f} %'.format(i,j) for i,j in zip(x, porcent)]

sort_legend = True
if sort_legend:
    patches, labels, dummy =  zip(*sorted(zip(patches, labels, y),
                                          key=lambda x: x[2],
                                          reverse=True))

plt.legend(patches, labels, loc='left center', bbox_to_anchor=(-0.1, 1.),
           fontsize=15)

plt.savefig('piechart.png', bbox_inches='tight')

**Análisis de blanceo:** como se puede observar en la partir gráfica presentada, los datos están desbalanceados. Claramente se destaca con un 80.2% el porcentaje de préstamos completamente pagos, en comparación con el 19.80% los no pagados.

A pesar de ello, era el resultado que esperabamos en función a los datos ya que podemos notar que refleja la verdadera relación entre los posibles valores de nuestro target. Por ello se decidió no descartar el dataset en cuestión.

Por otro lado, al tener conocimiento sobre este desbalanceo de los datos, podemos establecer a qué métricas no debemos darles tanta importancia, como es el caso de Accuracy, ya que ésta métrica presenta problemas con este tipo de datasets.


## Análisis de las variables de entrada:

In [ ]:
#VARIABLE GRADE
px.histogram(new_data.sort_index(),x='grade',color='loan_status',barmode='group')


*Análisis de la variable grade*: 

En función a la grafica presentada se pueden relizar las siguientes observaciones:
 - El mayor porcentaje de préstamos pagados se da en aquellos préstamos donde el grado tiene el valor B.
 
 
 - Como se esperaba, la mayor cantidad de préstamos está concentrada en aquellos clientes que estaban clasificados como A, B o C
 
 
 - También podemos observar que aquellos clientes clasificados como F y G fueron los que menos préstamos obtuvieron. 
 
 En estos casos se destaca la distribución balanceada entre aquellos prestatarios clasificados con el grado G. Aunque cabe destacar que no existe una gran diferencia entre los posibles valores del target también en aquellos clientes clasificados de grado F.

In [ ]:
#VARIABLE PURPOSE (CON HOT-ENCODER)

px.histogram(new_data.sort_index(),x='purpose',color='loan_status',barmode='group')

*Análisis de la variable purpose:*

En función a la grafica podemos destacar que:
- La mayor porcentaje de los préstamos solicitados tienen como próposito "consolidar obligaciones", es decir, aquellos préstamos que son obtenidos para pagar deudas. Además, es poca la cantidad que no son completamente pagados en comparación a los que son pagados.
- El porcentaje de préstamos solicitados con proposito educativo es muy pequeño, casi nulo podríamos decir.
- La mayor parte de préstamos solicitados para mejoras del hogar, como así también para pagar tarjetas de créditos, son pagados.

In [ ]:
#VARIABLE ANNUAL_INC
#Crea DataFrame para obtener rangos de ingresos anuales y poder graficar
range_inc=pd.DataFrame()
range_inc['loan_status']=new_data['loan_status']
range_inc['annual_inc']=new_data['annual_inc']
range_inc['ranges_inc']=pd.cut(x=range_inc['annual_inc'],bins=[0,25000,50000,75000,100000,125000,150000,1000000],labels=['Entre 0 y 25k','Entre 25k y 50k','Entre 50k y 75k','Entre 75k y 100k','Entre 100k y 125k','Entre 125k y 150k','Mayor a 150k'])


px.histogram(range_inc,x='ranges_inc',color='loan_status',barmode='group')




*Análisis de la variable "annual_inc":*

A partir del grafico podemos observar que los ingresos anuales de los prestarios se concentran en un rango de 50k y 70k. Aunque esto no parece afectar demasiado al target ya que si analizamos en función a los préstamos no pagados y, comparamos con aquellos clientes donde su ingreso anual varia entre los 25k y 50k, la diferencia es muy pequeña.


In [ ]:
#VARIABLE FUNDED_AMNT
#Crea DataFrame para obtener rangos de ingresos anuales y poder graficar
range_funded_amnt=pd.DataFrame()
range_funded_amnt['loan_status']=new_data['loan_status']
range_funded_amnt['funded_amnt']=new_data['funded_amnt']
range_funded_amnt['ranges_amnt']=pd.cut(x=range_funded_amnt['funded_amnt'],bins=[5000,10000,15000,20000,25000,30000,35000,40000],labels=['Menor a 5k','Entre 5k y 10k','Entre 10k y 15k','Entre 15k y 20k','Entre 20k y 25k','Entre 25k y 30k','Mayor a 35k'])


px.histogram(range_funded_amnt,x='ranges_amnt',color='loan_status',barmode='group')


*Análisis de la variable "funded_amnt":*

Respecto al gráfico de está variable podemos observar que la mayor parte de los préstamos otorgados se concentra en aquello con un monto menor a 5k , los de montos entre 5k y 10k aunque en menor medida.
Si hacemos un análisis respecto a como afecta está variable a nuestro target podemos deducir que el impacto de la misma no es grande en relación a los posibles valores que puede tomar "load_status".

In [ ]:
#VARIABLE EMP_LENGTH
px.histogram(new_data,x='emp_length',color='loan_status',barmode='group')

*Análisi de la variable "emp_length":*

Respecto a los valores de esta variable podemos observar claramente que el porcentaje de préstamos solicitados por aquellas personas que tienen el mismo empleo hace más de 10 años es mayor en relación a los demás.
Además, se descata que aquellos préstamos otorgados a personas que hace 3 años que poseen el mismo empleo presentan una tasa mayor de no pagados.